For any $0 < \epsilon < 1$ and any $n$ points in a high-dimensional space, there exists a linear map $f$ from the high-dimensional space to a low-dimensional space of dimension $d = O\left(\frac{\log n}{\epsilon^2}\right)$ such that for all pairs of points $p$ and $q$:

$$(1 - \epsilon) \|p - q\|^2 \leq \|f(p) - f(q)\|^2 \leq (1 + \epsilon) \|p - q\|^2$$

where $\|\cdot\|$ denotes the Euclidean norm. This means that the distances between the points are approximately preserved in the low-dimensional space, up to a factor of $(1 \pm \epsilon)$.




Here is some example code that ChatGPT gave to me, you can use it as a refrence, or make it work for the **Task 1** in TODO.

## TODO HERE: 
1. **(DONE)** First thing that should be done here, is taking all the data from the dataset in the form of `.jpg` files, and transforming it into the `numpy.array` of the shape $(N_{data}, 256,\ 256,\ 3)$ and further into the shape $(N_{data}, 256 \cdot 256 \cdot 3)$


In [3]:
# Presumably solving TODO 1

import os
import numpy as np
from PIL import Image

# Set the path to the data folders
input_folder = '../dataset/inputs_resized/flowers/'

# Set the size of the images
img_size = (256, 256)

# Load the images into a NumPy array
data = []
for category in os.listdir(input_folder):
    category_path = os.path.join(input_folder, category)
    print(category_path)
    for file in os.listdir(category_path):
        if file.endswith('.jpg'):
            img = Image.open(os.path.join(category_path, file)).resize(img_size)
            data.append(np.array(img))

data = np.array(data)

# Reshape the data into the desired shape
data = data.reshape(data.shape[0], -1)

print(data.shape)  # Output: (N_data, 196608)


../dataset/inputs_resized/flowers/daisy
../dataset/inputs_resized/flowers/rose
../dataset/inputs_resized/flowers/tulip
../dataset/inputs_resized/flowers/dandelion
../dataset/inputs_resized/flowers/sunflower
(4317, 196608)


In [138]:
import numpy as np
from sklearn.random_projection import johnson_lindenstrauss_min_dim, GaussianRandomProjection

# Compute the minimum number of dimensions required to preserve pairwise distances
eps = 0.1
n_samples, n_features = data.shape
n_components = johnson_lindenstrauss_min_dim(n_samples, eps=eps)

print(f"Minimum number of dimensions required: {n_components}")

# Project the data onto a lower-dimensional space using random projection
rp = GaussianRandomProjection(n_components=n_components, random_state=42)
data_rp = rp.fit_transform(data)

# Compute the pairwise distances between the original and projected data
dist_orig = np.zeros((n_samples, n_samples))
for i in range(n_samples):
    for j in range(i + 1, n_samples):
        dist_orig[i, j] = np.linalg.norm(data[i] - data[j])
        dist_orig[j, i] = dist_orig[i, j]

dist_rp = np.zeros((n_samples, n_samples))
for i in range(n_samples):
    for j in range(i + 1, n_samples):
        dist_rp[i, j] = np.linalg.norm(data_rp[i] - data_rp[j])
        dist_rp[j, i] = dist_rp[i, j]

# Compute the mean relative error between the pairwise distances of the original and projected data
# We can also look at the distribution of this error as a funciton of 
# N_reduced and epsilon, that may be interesting
mean_rel_error = np.mean(np.abs(dist_orig - dist_rp) / dist_orig)
print(f"Mean relative error: {mean_rel_error}")


Minimum number of dimensions required: 7174
Mean relative error: nan


In [132]:
data.shape

(400, 4096)